In [45]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pandas as pd

In [ ]:
## Load the trained model, scaler pickle,onehot

model=load_model("model.keras")

## load the encoder and scaler

with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file) 

with open("feature_columns.pkl", "rb") as f:
    feature_columns = pickle.load(f)       

In [47]:
## Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000  
}
input_df = pd.DataFrame([input_data])

In [48]:
## Encode categorical feature
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [49]:
geo_encoded = onehot_encoder_geo.transform(input_df[['Geography']]).toarray()

geo_df = pd.DataFrame(
    geo_encoded,
    columns=onehot_encoder_geo.get_feature_names_out(['Geography'])
)

input_df = pd.concat([input_df.drop('Geography', axis=1), geo_df], axis=1)

In [50]:
for col in feature_columns:
    if col not in input_df.columns:
        input_df[col] = 0
input_df = input_df[feature_columns]

In [51]:
input_scaled = scaler.transform(input_df)

In [53]:
prediction = model.predict(input_scaled)
print("Raw prediction:", prediction)

probability = prediction[0][0]
print(f"Churn Probability: {probability:.4f}")

if prediction[0][0] > 0.5:
    print("Customer is likely to churn ")
else:
    print("Customer is likely to stay ")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
Raw prediction: [[0.02084]]
Churn Probability: 0.0208
Customer is likely to stay 
